In [1]:
from matplotlib import pyplot as plt
import numpy as np
from time import time
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info.operators.base_operator import BaseOperator
from qiskit.providers.aer.pulse_new.de.DE_Problems import BMDE_Problem
from qiskit.providers.aer.pulse_new.de.DE_Solvers import BMDE_Solver
from qiskit.providers.aer.pulse_new.de.DE_Options import DE_Options
from qiskit.providers.aer.pulse_new.models.signals import VectorSignal, Constant, Signal
from qiskit.providers.aer.pulse_new.models.operator_models import OperatorModel

X = Operator.from_label('X')
Y = Operator.from_label('Y')
Z = Operator.from_label('Z')

# 1. Basic BMDE Problem/Solver interface

A `BMDE_Problem` represents a differential equation of the form $\dot{y}(t) = G(t) y(t)$, where currently it is assumed that $G(t)$ is specified by an `OperatorModel`. It is the core type of BMDE specification from which others will be built. E.g. the Schrodinger equation for a state vector will be a subclass of `BMDE_Problem`, accepting a `HamiltonianModel` and converting it into the proper generator.

Construct an `OperatorModel` for the generator:

In [2]:
w = 5.
r = 0.02
signals = [Constant(1.), Signal(1., w)]
operators = [-1j * 2 * np.pi * w * Z/2, -1j * 2 * np.pi * r * X/2]

generator = OperatorModel(operators=operators, signals=signals)

Construct the problem with the generator, initial time, and initial state.

In [3]:
y0 = np.array([1., 0.])

bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0)

Construct the solver with the problem and some options.

In [4]:
options = DE_Options(atol=1e-8, rtol=1e-8)
solver = BMDE_Solver(bmde_problem, options=options)

Integrate up to some time. Here the time is chosen to implement a $\pi$-pulse.

In [5]:
solver.integrate(1. / r)

Look at the state.

In [6]:
np.abs(solver.y)**2

array([2.49975534e-07, 9.99999808e-01])

# 2. Specifying solving frames

When solving a BMDE, it can be numerically advantageous to solve in a different frame. A frame to solve the system in can be specified when constructing a `BMDE_Problem` via the keyword argument `frame_operator`. The default value, `frame_operator='auto'`, lets the `BMDE_Problem` class automatically choose the frame to solve in. Currently, it will automatically choose the anti-Hermitian part of the drift. We can set different values for `frame_operator` and see how long it takes to solve the system.

Setting `frame_operator=None` forces the problem to be solved in the frame it is specified in. 

In [7]:
bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0, frame=None)
options = DE_Options(atol=1e-10, rtol=1e-10)
solver = BMDE_Solver(bmde_problem, options=options)

print("Time to integrate in 'lab' frame")
print("--------------------------------")
%time solver.integrate(1. / r)
print("Final state:")
print("------------")
print(solver.y)

Time to integrate in 'lab' frame
--------------------------------
CPU times: user 6.87 s, sys: 23.2 ms, total: 6.9 s
Wall time: 6.88 s
Final state:
------------
[1.96307349e-07+4.99998576e-04j 2.59829658e-12-9.99999834e-01j]


Next, solve the system in the automatically determined frame

In [8]:
bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0, frame='auto')
options = DE_Options(atol=1e-10, rtol=1e-10)
solver = BMDE_Solver(bmde_problem, options=options)

print("Time to integrate in 'drift' frame")
print("---------------------------------")
%time solver.integrate(1. / r)
print("Final state:")
print("------------")
print(solver.y)

Time to integrate in 'drift' frame
---------------------------------
CPU times: user 3.51 s, sys: 8.73 ms, total: 3.52 s
Wall time: 3.51 s
Final state:
------------
[1.96347361e-07+5.00000436e-04j 6.15066315e-13-9.99999875e-01j]


Above we see a speed up of nearly a factor of $2$. The scale of the time differences largely depends on parameters of the system.

# 3. Solving with cutoff frequencies (Rotating wave approximation)

In addition to specifying frame, we may also specify a cutoff frequency. The standard cutoff frequency for this model, the rotating wave approximation, is `2 * w`.

In [9]:
bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0, cutoff_freq=2*w)
options = DE_Options(atol=1e-10, rtol=1e-10)
solver = BMDE_Solver(bmde_problem, options=options)

print("Time to integrate in 'drift' frame with RWA")
print("-------------------------------------------")
%time solver.integrate(1. / r)
print(solver.y)

Time to integrate in 'drift' frame with RWA
-------------------------------------------
CPU times: user 25.6 ms, sys: 345 µs, total: 26 ms
Wall time: 25.8 ms
[-4.66628125e-12+1.68729864e-15j -8.10197971e-14-1.00000000e+00j]


Observe: solving with a cutoff frequency increases the speed of solving dramatically, albeit at the expense of a loss of accuracy. Again, the scale of the speed gains and accuracy loss will depend on the parameters of the model.

# 4. Specifying different numerical methods

The default method is `scipy`'s RK45, but we can specify different underlying `DE_Methods` using the `method` keyward argument in `DE_Options`. E.g. `Expm` specifies a matrix exponentiation-based fixed-step solver, requiring specification of a max step size.

Note that in this size the generator in the frame is actually constant with an RWA cutoff of `2*w`, so we can set the `max_dt` to the whole interval we wish to simulate.

In [10]:
bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0, cutoff_freq=2*w)
options = DE_Options(method='Expm', max_dt=(1.0 / r ))
solver = BMDE_Solver(bmde_problem, options=options)

print("Time to integrate in 'drift' frame with RWA, using single matrix exponential")
print("----------------------------------------------------------------------------")
%time solver.integrate(1. / r)
print(solver.y)

Time to integrate in 'drift' frame with RWA, using single matrix exponential
----------------------------------------------------------------------------
CPU times: user 4.3 ms, sys: 1.74 ms, total: 6.04 ms
Wall time: 1.95 ms
[ 1.11022302e-16+8.92110402e-30j -8.03541615e-14-1.00000000e+00j]
